In [ ]:
!source /home/nikhil/Desktop/repo/KamleshThesis/env/bin/activate

In [ ]:
import os

import keras.layers as layers
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from keras.callbacks import ModelCheckpoint
from sklearn.utils import shuffle
from tensorflow.keras import Model

from deepface.basemodels.ArcFace import loadModel
from deepface.commons import functions
import matplotlib.pyplot as plt


In [ ]:
model = loadModel()

model=ArcFace
ArcFace.summary()

In [ ]:
# Define the Contrastive Loss
def contrastive_loss(y, preds, margin=1):
    y = tf.cast(y, preds.dtype)
    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)

    return loss

In [ ]:
# Distance Layer
class DistanceLayer(layers.Layer):
    """
    This layer is responsible for computing the distance
    between the embeddings
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, compare):
        sum_squared = K.sum(K.square(anchor - compare), axis=1, keepdims=True)
        return K.sqrt(K.maximum(sum_squared, K.epsilon()))

In [ ]:
#decide input shape
input_shape_x, input_shape_y = functions.find_input_shape(model)
t_shape = (input_shape_x, input_shape_y, 3)

anchor_input = layers.Input(name="anchor", shape=t_shape)
compare_input = layers.Input(name="compare", shape=t_shape)

distances = DistanceLayer()(
    model(anchor_input),
    model(compare_input),
)

outputs = layers.Dense(1, activation = "sigmoid") (distances)

model = Model(
    inputs=[anchor_input, compare_input], outputs=outputs,
    name = "ArcFace_Training"
)

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

## Load DataSet

In [ ]:
def get_img(img_path, input_shape_y = input_shape_y, input_shape_x = input_shape_x, enforce_detection = True
		, detector_backend = 'opencv', align = True, normalization = 'base'):
	'''Returns Image when given the following details'''
	#detect and align
	img = functions.preprocess_face(img = img_path
		, target_size=(input_shape_y, input_shape_x)
		, enforce_detection = enforce_detection
		, detector_backend = detector_backend
		, align = align)

	#custom normalization
	img = functions.normalize_input(img = img, normalization = normalization)
	return img

In [ ]:
# Get the data directory paths
data_folder = "/home/nikhil/Desktop/repo/KamleshThesis/data/LFW/lfw/"
img_type = ['jpg']

In [ ]:
name_list = os.listdir(data_folder)
name_list = name_list[:300]
face_labels = []
face_images = []

print("Processing ", len(name_list), " Folders\n\n")
for fold in name_list:
    per_path = os.path.join(data_folder, fold)
    if os.path.isdir(per_path):
        for img in os.listdir(per_path):
            img_path = os.path.join(per_path, img)
            if os.path.isfile(img_path) and img[-3:].lower() in img_type:
                try:
                    img_arr = get_img(img_path)
                    face_labels.append(name_list.index(fold))
                    face_images.append(img_arr)
                except:
                    print('Cannot load '+fold)


In [ ]:
img_len = len(face_images)
if img_len == 1:
    print("removing Extra")
    face_images = face_images[0]

if len(face_images[0]) != input_shape_x:
    for index, face in enumerate(face_images):
        face_images[index] = face[0]

if img_len == len(face_labels):
    print('success')
else:
    print('Failiure')

In [ ]:
print(len(face_images[0][0]))
face_images

In [ ]:
# Function to generate image pairs
def generate_image_pairs(images, labels):
    # Generate index for each label
    unique_labels = np.unique(labels)
    label_wise_indices = dict()
    for label in unique_labels:
        label_wise_indices.setdefault(label,
                                      [index for index, curr_label in enumerate(labels) if
                                       label == curr_label])
    
    # Generate image pairs and labels
    pair_images = []
    pair_labels = []
    for index, image in enumerate(images):
        pos_indices = label_wise_indices.get(labels[index])
        pos_image = images[np.random.choice(pos_indices)]
        pair_images.append((image, pos_image))
        pair_labels.append(1)

        neg_indices = np.where(labels != labels[index])
        neg_image = images[np.random.choice(neg_indices[0])]
        pair_images.append((image, neg_image))
        pair_labels.append(0)
        
    return np.array(pair_images), np.array(pair_labels)

In [ ]:
images_dataset, labels_dataset = generate_image_pairs(face_images, face_labels)
images_dataset, labels_dataset = shuffle(images_dataset, labels_dataset)

In [ ]:
images_dataset.shape

In [ ]:
history = model.fit(
    [images_dataset[:, 0, :], images_dataset[:, 1, :]], 
    labels_dataset,
    epochs=10, 
    validation_split = 0.2, 
    batch_size = 64, 
    callbacks=[
        ModelCheckpoint(
            'model.hdf5',
            verbose=1, 
            save_best_only=True
        )
    ]
)

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.legend(["Loss", "Validation Loss", "Accuracy", "Validation Accuracy"])

In [ ]:
def test_visualize(images, n = 5):
    """ Visualize a few images """

    def show(ax, image):
        ax.imshow(image)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    fig = plt.figure(figsize=(9, 9)) 
    axs = fig.subplots(1, n)
    for i in range(n):
        show(axs[i], images[i])


In [ ]:
import random


In [ ]:
test_indices = [0, 1, 2, 3, 4]
test_index = random.sample(range(0, 50), 1)[0]
test_image = face_images[test_index]

compare_images = []
for i in range(5):
    index = random.sample(range(i * 10, (i + 1) * 10), 1)[0]
    image = face_images[index]
    compare_images.append(image)
plt.imshow(test_image)